In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from sklearn.feature_extraction.text import CountVectorizer
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re

In [29]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

print(data)

                                                    text sentiment
0      RT @NancyLeeGrahn: How did everyone feel about...   Neutral
1      RT @ScottWalker: Didn't catch the full #GOPdeb...  Positive
2      RT @TJMShow: No mention of Tamir Rice and the ...   Neutral
3      RT @RobGeorge: That Carly Fiorina is trending ...  Positive
4      RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  Positive
...                                                  ...       ...
13866  RT @cappy_yarbrough: Love to see men who will ...  Negative
13867  RT @georgehenryw: Who thought Huckabee exceede...  Positive
13868  RT @Lrihendry: #TedCruz As President, I will a...  Positive
13869  RT @JRehling: #GOPDebate Donald Trump says tha...  Negative
13870  RT @Lrihendry: #TedCruz headed into the Presid...  Positive

[13871 rows x 2 columns]


In [30]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:3: SyntaxWarning: invalid escape sequence '\s'
/var/folders/ky/17f_m8v91pz3wc_l24h7y22h0000gn/T/ipykernel_22396/3498670558.py:3: SyntaxWarning: invalid escape sequence '\s'
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
/var/folders/ky/17f_m8v91pz3wc_l24h7y22h0000gn/T/ipykernel_22396/3498670558.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  row[0] = row[0].replace('rt',' ')
/var/folders/ky/17f_m8v91pz3wc_l24h7y22h0000gn/T/ipykernel_22396/3498670558.py:9: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[0] = row[0].replace(

4472
16986


In [31]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/Users/jenusanyogarajah/Google-Review-Helper/SentAnalysis/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [32]:

Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


In [33]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
225/225 - 5s - 23ms/step - accuracy: 0.8114 - loss: 0.4494
Epoch 2/10
225/225 - 5s - 21ms/step - accuracy: 0.8625 - loss: 0.3297
Epoch 3/10
225/225 - 5s - 20ms/step - accuracy: 0.8843 - loss: 0.2856
Epoch 4/10
225/225 - 5s - 20ms/step - accuracy: 0.8957 - loss: 0.2581
Epoch 5/10
225/225 - 5s - 20ms/step - accuracy: 0.9053 - loss: 0.2310
Epoch 6/10
225/225 - 5s - 21ms/step - accuracy: 0.9154 - loss: 0.2130
Epoch 7/10
225/225 - 5s - 20ms/step - accuracy: 0.9256 - loss: 0.1892
Epoch 8/10
225/225 - 5s - 22ms/step - accuracy: 0.9299 - loss: 0.1775
Epoch 9/10
225/225 - 5s - 20ms/step - accuracy: 0.9328 - loss: 0.1603
Epoch 10/10
225/225 - 5s - 20ms/step - accuracy: 0.9399 - loss: 0.1454


In [34]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

64/64 - 0s - 7ms/step - accuracy: 0.8295 - loss: 0.5620
score: 0.56
acc: 0.83


In [35]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 83ms/step
1/1 - 0s - 10ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 10ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 10ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 9ms/step
1/1 - 0s - 8ms/step
1/1 - 0s - 8ms/

In [49]:
twt = ["it was good"]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print((sentiment[0]))
    print("negative")
elif (np.argmax(sentiment) == 1):
    print((sentiment[1]))
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0  14  22 137]]
1/1 - 0s - 13ms/step
0.8274129
positive
